In [1]:
pip install pyspark

     |████████████████████████████████| 212.4MB 85kB/s 
     |████████████████████████████████| 204kB 23.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=104a69ddde532024503d61010b5eb813db838638120c2fd6ba078da5d1926198
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [2]:
# Definindo bibliotecas
from pyspark.sql import SparkSession # Criando a sessão
from pyspark.sql.functions import explode #separa os elementos de uma linha em multiplas linhas
from pyspark.sql.functions import split #divide as linhas

spark = SparkSession\
    .builder \
    .appName('StructuredNetworkWordCount') \
    .getOrCreate()

In [4]:
#Cria o dataFrame que será responsavel por ler cada uma das linhas recebidas através do localhost e porta 9999
# define a fonte (source) de dados

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Divide as linhas recebidas em cada palavra
words = lines.select(
    explode(
        split(lines.value, " ")
    ).alias("word")
)

# Cria o novo dataframe a ser responsável por contar a quantidade de palavras digitadas
wordCounts = words.groupBy("word").count()

In [6]:
lines.isStreaming

True

In [8]:
nc -lk 9999 # precisa ter o nc instalado (linux)

SyntaxError: ignored

In [13]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado
query = wordCounts \
    .writeStream \
    .outputModel("update") \
    .start()

query.awaitTermination() #aguardar até que a "streaming query" termine

AttributeError: ignored